In [1]:
import pandas as pd


In [2]:
df_21 = pd.read_csv("../data_files/21zpallagi.csv")
df_20 = pd.read_csv("../data_files/20zpallagi.csv")
df_19 = pd.read_csv("../data_files/19zpallagi.csv")
df_20 = pd.read_csv("../data_files/20zpallagi.csv")
df_18 = pd.read_csv("../data_files/18zpallagi.csv")
df_17 = pd.read_csv("../data_files/17zpallagi.csv")
df_16 = pd.read_csv("../data_files/16zpallagi.csv")
df_15 = pd.read_csv("../data_files/15zpallagi.csv")
df_14 = pd.read_csv("../data_files/14zpallagi.csv")
df_13 = pd.read_csv("../data_files/13zpallagi.csv")
df_12 = pd.read_csv("../data_files/12zpallagi.csv")

In [3]:
#creating a function to drop unwanted columns and unwanted rows for all dataframes
def drop_unwanted(df):
    #put columns in a list
    columns_to_keep = [
    'STATE', 'zipcode', 'agi_stub', 'N1', 'mars1', 'MARS2', 'MARS4',
    'A00100', 'A02650', 'N01000', 'A01000', 'N02910', 'A02910',
    'N04470', 'A04470', 'N18300', 'A18300',
    'N19700', 'A19700'
    ]
    #drop unwanted columns
    existing_columns = [col for col in columns_to_keep if col in df.columns]
    df = df[existing_columns]

    # Print missing columns (columns in columns_to_keep that are not in df.columns)
    missing_columns = [col for col in columns_to_keep if col not in df.columns]
    if missing_columns:
        print(f"Skipping non-existent columns: {missing_columns}")

    return df

In [4]:
#function to rename the other df columnms
def rename_cols(df):
    #variable to map old names to new names
    column_map = {
    'STATE': 'State',
    'zipcode': 'Zip',
    'agi_stub': 'AGI_Range',
    'N1': 'Returns',
    'mars1': 'Single',
    'MARS2': 'Joint',
    'MARS4': 'Hd_of_Hshld',
    'A00100': 'AGI_Amt',
    'A02650': 'Total_Income',
    'N01000': 'Cap_Gain_Returns',
    'A01000': 'Cap_Gain_Amt',
    'N02910': 'Std_Ded_Charity_Returns',
    'A02910': 'Std_Ded_Charity_Amt',
    'N04470': 'Itemized_Ded_Returns',
    'A04470': 'Itemized_Ded_Amt',
    'N18300': 'Total_Taxes_Returns',
    'A18300': 'Total_Taxes_Amt',
    'N19700': 'Charitable_Returns',
    'A19700': 'Charitable_Amt'
    }
    #rename
    try:
        # Rename columns based on the mapping dictionary (if columns exist in the DataFrame)
        df = df.rename(columns=column_map)
    except KeyError as e:
        missing_columns = [col for col in column_map.keys() if col not in df.columns]
        print(f"Skipping non-existent columns during renaming: {missing_columns}")

    
    columns_to_multiply = ['AGI_Amt', 'Standar_Ded_Amt', 'Itemized_Ded_Amt', 'Total_Taxes_Amt', 'Charitable_Amt']
    
    try:
        # Multiply monetary columns by 1000 (if columns exist in the DataFrame)
        df[columns_to_multiply] = df[columns_to_multiply] * 1000
    except KeyError as e:
        missing_columns = [col for col in columns_to_multiply if col not in df.columns]
        print(f"Skipping non-existent columns: {missing_columns}")

    return df
    


    

In [5]:
#function to route other df's through other functions

def route_through_2(df):

    processed_df = drop_unwanted(df)
    df = rename_cols (processed_df)
    return df
    

In [6]:
#list comprehension to run df's through functions

df_21,df_20,df_19,df_18,df_17,df_16,df_15,df_14,df_13,df_12 = [route_through_2(df) for df in [df_21,df_20,df_19,df_18,df_17,df_16,df_15,df_14,df_13,df_12]]


Skipping non-existent columns: ['Standar_Ded_Amt']
Skipping non-existent columns: ['Standar_Ded_Amt']
Skipping non-existent columns: ['N02910', 'A02910']
Skipping non-existent columns: ['Standar_Ded_Amt']
Skipping non-existent columns: ['N02910', 'A02910']
Skipping non-existent columns: ['Standar_Ded_Amt']
Skipping non-existent columns: ['N02910', 'A02910']
Skipping non-existent columns: ['Standar_Ded_Amt']
Skipping non-existent columns: ['N02910', 'A02910']
Skipping non-existent columns: ['Standar_Ded_Amt']
Skipping non-existent columns: ['N02910', 'A02910']
Skipping non-existent columns: ['Standar_Ded_Amt']
Skipping non-existent columns: ['N02910', 'A02910']
Skipping non-existent columns: ['Standar_Ded_Amt']
Skipping non-existent columns: ['mars1', 'N02910', 'A02910']
Skipping non-existent columns: ['Standar_Ded_Amt']
Skipping non-existent columns: ['agi_stub', 'mars1', 'A02650', 'N02910', 'A02910']
Skipping non-existent columns: ['Standar_Ded_Amt']


In [18]:
def calculate_average_charitable_contributions(df):

    # Attempt to process standard deduction returns
    try:
        std_ded_df = df[df['Std_Ded_Charity_Returns'] > 0]  # Filter for standard deduction returns
        std_ded_avg = std_ded_df.groupby(['State', 'AGI_Range'])['Charitable_Amt'].mean().reset_index()
        std_ded_avg['Deduction_Type'] = 'Standard'  # Add column for deduction type
        std_ded_avg['Average_Charitable_Contribution'] = std_ded_avg['Charitable_Amt']  # Add column for average contribution
    except KeyError:
        print("Warning: Standard deduction columns not found. Skipping standard deduction calculations.")
        std_ded_avg = pd.DataFrame()  # Create an empty DataFrame for standard deduction
    
    # Attempt to process itemized deduction returns
    try:
        itemized_ded_df = df[df['Itemized_Ded_Returns'] > 0]  # Filter for itemized deduction returns
        itemized_ded_avg = itemized_ded_df.groupby(['State', 'AGI_Range'])['Charitable_Amt'].mean().reset_index()
        itemized_ded_avg['Deduction_Type'] = 'Itemized'  # Add column for deduction type
        itemized_ded_avg['Average_Charitable_Contribution'] = itemized_ded_avg['Charitable_Amt']  # Add column for average contribution
    except KeyError:
        print("Warning: Itemized deduction columns not found. Skipping itemized deduction calculations.")
        itemized_ded_avg = pd.DataFrame()  # Create an empty DataFrame for itemized deduction

    # Concatenate results while filtering out empty DataFrames
    results = []
    if not std_ded_avg.empty:
        results.append(std_ded_avg)
    if not itemized_ded_avg.empty:
        results.append(itemized_ded_avg)
    
    if results:
        avg_charitable_contributions = pd.concat(results, ignore_index=True)
    else:
        avg_charitable_contributions = pd.DataFrame()  # Return empty DataFrame if no valid results
    
    return avg_charitable_contributions


In [19]:
def calculate_average_charitable_contribution(df):
    
    # Filter DataFrame for standard and itemized returns
    std_ded_df = df[df['Std_Ded_Charity_Returns'] > 0]  # Filter for standard deduction returns
    itemized_ded_df = df[df['Itemized_Ded_Returns'] > 0]  # Filter for itemized deduction returns

    # Group by state and age range, and calculate average charitable contributions
    avg_charitable_contributions = pd.DataFrame()
    
    # Calculate average charitable contributions for standard deduction returns
    std_ded_avg = std_ded_df.groupby(['State', 'AGI_Range'])['Charitable_Amt'].mean().reset_index()
    std_ded_avg['Deduction_Type'] = 'Standard'  # Add column for deduction type
    std_ded_avg['Average_Charitable_Contribution'] = std_ded_avg['Charitable_Amt']  # Add column for average contribution
    avg_charitable_contributions = avg_charitable_contributions.append(std_ded_avg, ignore_index=True)
    
    # Calculate average charitable contributions for itemized deduction returns
    itemized_ded_avg = itemized_ded_df.groupby(['State', 'AGI_Range'])['Charitable_Amt'].mean().reset_index()
    itemized_ded_avg['Deduction_Type'] = 'Itemized'  # Add column for deduction type
    itemized_ded_avg['Average_Charitable_Contribution'] = itemized_ded_avg['Charitable_Amt']  # Add column for average contribution
    avg_charitable_contributions = avg_charitable_contributions.append(itemized_ded_avg, ignore_index=True)
    
    return avg_charitable_contributions


In [25]:
#acc_16=calculate_average_charitable_contributions(df_16)
acc_16.head(40)

,State,AGI_Range,Charitable_Amt,Deduction_Type,Average_Charitable_Contribution
0,AK,1,201.658537,Itemized,201.658537
1,AK,2,553.309091,Itemized,553.309091
2,AK,3,911.238806,Itemized,911.238806
3,AK,4,1342.854545,Itemized,1342.854545
4,AK,5,3569.630769,Itemized,3569.630769
5,AK,6,5049.302326,Itemized,5049.302326
6,AL,1,474.082569,Itemized,474.082569
7,AL,2,1249.055659,Itemized,1249.055659
8,AL,3,1541.225806,Itemized,1541.225806
9,AL,4,1642.200418,Itemized,1642.200418


In [27]:
acc_20=calculate_average_charitable_contributions(df_20)
acc_20.head(40)

,State,AGI_Range,Charitable_Amt,Deduction_Type,Average_Charitable_Contribution
0,AK,1,20.901961,Standard,20.901961
1,AK,2,192.800000,Standard,192.800000
2,AK,3,325.963636,Standard,325.963636
3,AK,4,598.347826,Standard,598.347826
4,AK,5,2286.538462,Standard,2286.538462
5,AK,6,6392.342857,Standard,6392.342857
6,AL,1,60.201077,Standard,60.201077
7,AL,2,363.378330,Standard,363.378330
8,AL,3,537.639640,Standard,537.639640
9,AL,4,542.893281,Standard,542.893281
